In [12]:
import pandas as pd
review = pd.read_csv("final_total_reviews.csv")
review.head(20)

,product_id,user_id,user_keywords,user_rating,review_text
0,M1,겨울브라이트21호,건성,5,건조할때 사용하기 좋아요 사용전 충분히 흔들어서 사용해야 유분기가 덜합니다. 속건조...
1,M1,재이맘2,지성,5,세안 후 바로 발라도 밀림 없이 잘 흡수됐어요. 건조로 인한 당김이 눈에 띄게 줄어...
2,M1,엔코,복합성 | 민감성,5,달바가 보습으로 유명하잖아요! 확실히 유명한 미스트라 그런지 역시 보습감 최고입니다
3,M1,서유니맘2,지성,5,겉으로 보면 뭔가 디게 오일리한 것 같은데 막상 사용해보면 그 정도는 아니네요.
4,M1,소보소보,트러블성,5,지인픽 선물입니다 달바 용기 재활용이 불가한데 재활용 용기로의 전환 고민해주심 좋겠어요
5,M1,1k,지성,5,건조할 때마다 뿌리면 즉각 광채! 미스트는 역시 달바가 정답인 것 같습니다
6,M1,노냄,민감성,5,퍼스트 스프레이 세럼 듀오로 넉넉하게! 피부 속부터 차오르는 촉촉함이 좋아요
7,M1,노냄,민감성,5,NO.1 미스트 세럼 100ml 2개 기획! 건조한 피부에 영양 오일 보습막을 형성해요.
8,M1,eunbi3804,복합성 | 민감성,5,달바 퍼스트 스프레이 세럼은 미세 안개 분사로 피부에 고르게 밀착돼요. 세안 후 첫...
9,M1,ㅣ김여름ㅣ,지성,5,💛 달바 퍼스트 스프레이 세럼 💛 구성 : 달바 퍼스트 스프레이 세럼 100ml 2...


In [ ]:
!pip -q install -U safetensors transformers

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm.auto import tqdm

df = pd.read_csv("final_total_reviews.csv")
texts = df["review_text"].fillna("").astype(str).tolist()

MODEL_ID = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
device = 0 if torch.cuda.is_available() else -1

tok = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    use_safetensors=True  # ✅ 핵심: bin 말고 safetensors로 로드
)

clf = pipeline("sentiment-analysis", model=model, tokenizer=tok, device=device)

batch_size = 32
outs = []
for i in tqdm(range(0, len(texts), batch_size)):
    outs.extend(clf(texts[i:i+batch_size], truncation=True, max_length=256))

df["sentiment_raw_label"] = [o["label"] for o in outs]
df["sentiment_score"] = [o["score"] for o in outs]
df["sentiment"] = df["sentiment_raw_label"].str.lower()

df.to_csv("final_total_reviews_with_sentiment.csv", index=False)
df.head()


config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

ValueError: Could not load model cardiffnlp/twitter-xlm-roberta-base-sentiment with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.xlm_roberta.modeling_xlm_roberta.XLMRobertaForSequenceClassification'>). See the original errors:

while loading with AutoModelForSequenceClassification, an error is thrown:
Traceback (most recent call last):
  File "/Users/jeong-yeon-u/anaconda3/envs/gnn/lib/python3.11/site-packages/transformers/pipelines/base.py", line 293, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jeong-yeon-u/anaconda3/envs/gnn/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py", line 604, in from_pretrained
    return model_class.from_pretrained(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jeong-yeon-u/anaconda3/envs/gnn/lib/python3.11/site-packa

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [13]:
review[['user_keywords','user_rating','review_text']].head(20)

,user_keywords,user_rating,review_text
0,건성,5,건조할때 사용하기 좋아요 사용전 충분히 흔들어서 사용해야 유분기가 덜합니다. 속건조...
1,지성,5,세안 후 바로 발라도 밀림 없이 잘 흡수됐어요. 건조로 인한 당김이 눈에 띄게 줄어...
2,복합성 | 민감성,5,달바가 보습으로 유명하잖아요! 확실히 유명한 미스트라 그런지 역시 보습감 최고입니다
3,지성,5,겉으로 보면 뭔가 디게 오일리한 것 같은데 막상 사용해보면 그 정도는 아니네요.
4,트러블성,5,지인픽 선물입니다 달바 용기 재활용이 불가한데 재활용 용기로의 전환 고민해주심 좋겠어요
5,지성,5,건조할 때마다 뿌리면 즉각 광채! 미스트는 역시 달바가 정답인 것 같습니다
6,민감성,5,퍼스트 스프레이 세럼 듀오로 넉넉하게! 피부 속부터 차오르는 촉촉함이 좋아요
7,민감성,5,NO.1 미스트 세럼 100ml 2개 기획! 건조한 피부에 영양 오일 보습막을 형성해요.
8,복합성 | 민감성,5,달바 퍼스트 스프레이 세럼은 미세 안개 분사로 피부에 고르게 밀착돼요. 세안 후 첫...
9,지성,5,💛 달바 퍼스트 스프레이 세럼 💛 구성 : 달바 퍼스트 스프레이 세럼 100ml 2...


In [8]:
review['user_rating'].value_counts()   

user_rating
5    56940
4     2944
3      895
1      162
2      159
Name: count, dtype: int64

In [14]:
"""
피부타입별 제품별 평균 평점 계산 (간단 버전)
"""

import pandas as pd

# 데이터 로드
review_df = pd.read_csv('final_total_reviews.csv')

# 피부타입 전처리
review_df['skin_types'] = review_df['user_keywords'].apply(
    lambda x: [s.strip() for s in str(x).split('|')] if pd.notna(x) else []
)

# 피부타입 목록
skin_types = ['건성', '민감성', '복합성', '약건성', '지성', '트러블성', '중성']

# 제품별 피부타입별 평균 평점 계산
def get_product_skin_ratings(product_id):
    """특정 제품의 피부타입별 평균 평점"""
    product_reviews = review_df[review_df['product_id'] == product_id]
    
    result = {}
    for skin_type in skin_types:
        # 해당 피부타입 리뷰 필터
        skin_reviews = product_reviews[
            product_reviews['skin_types'].apply(lambda x: skin_type in x)
        ]
        
        if len(skin_reviews) > 0:
            # 평균 평점 계산
            avg_rating = skin_reviews['user_rating'].mean()
            result[skin_type] = round(avg_rating, 2)
        else:
            result[skin_type] = None
    
    return result

# 예시 사용
product_id = review_df['product_id'].iloc[0]
ratings = get_product_skin_ratings(product_id)

print(f"제품 {product_id}의 피부타입별 평균 평점:")
for skin_type, rating in ratings.items():
    if rating:
        print(f"  {skin_type}: {rating}점")
    else:
        print(f"  {skin_type}: 리뷰 없음")

# 전체 제품에 대해 계산
all_results = []
for product_id in review_df['product_id'].unique():
    ratings = get_product_skin_ratings(product_id)
    ratings['product_id'] = product_id
    all_results.append(ratings)

# DataFrame으로 변환
result_df = pd.DataFrame(all_results)
print(f"\n전체 결과 (첫 10개):")
print(result_df.head(10))

# CSV 저장
result_df.to_csv('skin_type_ratings_simple.csv', index=False, encoding='utf-8-sig')
print("\n저장 완료: skin_type_ratings_simple.csv")

제품 M1의 피부타입별 평균 평점:
  건성: 4.74점
  민감성: 4.89점
  복합성: 4.93점
  약건성: 4.5점
  지성: 4.89점
  트러블성: 5.0점
  중성: 리뷰 없음

전체 결과 (첫 10개):
     건성   민감성   복합성  약건성    지성  트러블성    중성 product_id
0  4.74  4.89  4.93  4.5  4.89  5.00   NaN         M1
1  4.82  4.62  4.30  5.0  4.73  5.00  5.00         M2
2  4.85  4.70  4.84  5.0  4.57  5.00  5.00         M3
3  4.77  4.57  4.78  NaN  4.64  5.00  5.00         M4
4  4.45  4.44  4.46  NaN  4.41  3.50  4.71         M5
5  4.82  4.86  4.84  4.5  4.92  5.00  5.00         M8
6  5.00  4.95  4.86  5.0  4.88  5.00  3.00         M9
7  4.56  4.90  4.71  5.0  4.77  4.75  4.50        M10
8  5.00  5.00  5.00  5.0  5.00  5.00  5.00        M11
9  4.81  4.76  4.86  NaN  4.82  5.00  5.00        M12

저장 완료: skin_type_ratings_simple.csv


In [9]:
product=pd.read_csv("final_products.csv")
product.head(10)

,product_id,category,brand,product_name,ingredients,product_rating
0,L1,로션,온그리디언츠,스킨 베리어 카밍 로션 이엑스,"['글리세린', '페닐트라이메티콘', '알지닌', '하이드록시에틸우레아', '하이드...",5.0
1,L2,로션,VT,리들샷 포맨 올인원 100 엠플러스,"['글리세린', '나이아신아마이드', '실리카', '베타인', '하이드록시아세토페논...",5.0
2,L4,로션,에스트라,아토베리어365 로션,"['글리세린', '스쿠알란', '펜타에리스리틸테트라아이소스테아레이트', '베타인',...",4.9
3,L5,로션,아이오페,맨 올데이 퍼펙트 올인원,"['글리세린', '베타인', '나이아신아마이드', '실리카', '다이메티콘올', '...",4.9
4,L6,로션,아이디얼포맨,퍼펙트 올인원,"['글리세린', '에틸헥실스테아레이트', '나이아신아마이드', '베타인', '다시마...",4.9
5,L8,로션,닥터지,레드 블레미쉬 포 맨 진정 올인원,"['나이아신아마이드', '카프릴릴메티콘', '글리세린', '하이드로제네이티드폴리',...",4.9
6,L9,로션,아벤느,트릭세라 뉴트리 플루이드 로션 더블 윤광 광채 크림,"['온천수', '글리세린', '사이클로메치콘', '달맞이꽃오일', '피이지-100스...",4.8
7,L10,로션,라운드랩,포 맨 1025 독도 올인원 젤로션,"['베타인', '달맞이꽃꽃추출물', '대왕소나무잎추출물', '칡뿌리추출물', '당느...",4.9
8,L11,로션,한율,어린쑥 수분진정 플루이드,"['글리세린', '황해쑥잎수', '피씨에이다이메티콘', '천궁수', '왜당귀뿌리수'...",4.8
9,L13,로션,우르오스,올인원 스킨밀크,"['글리세린', '하이드로제네이티드폴리', '아데노신포스페이트', '하이드록시프로필...",4.8


In [10]:
product['product_rating'].value_counts()

product_rating
4.8    273
4.9    158
4.7    123
5.0     29
4.6     19
4.5      9
Name: count, dtype: int64